# Data exploration

In [27]:
import org.apache.spark

import org.apache.spark


### Schema definitions

In [28]:
import org.apache.spark.sql.types._
import java.sql.Timestamp

val reviewSchema = StructType(
  Seq(
    StructField("user_id",  StringType,            nullable = true),
    StructField("name",     StringType,            nullable = true),
    StructField("time",     LongType,              nullable = false),
    StructField("rating",   DoubleType,            nullable = true),
    StructField("text",     StringType,            nullable = true),
    StructField("pics",     ArrayType(StringType), nullable = true),
    StructField("resp",     StructType(
      Seq(
        StructField("time", LongType,              nullable = false),
        StructField("text", StringType,            nullable = true)
      )
    ),                                             nullable = true),
    StructField("gmap_id",  StringType,            nullable = false),
  )
)

case class Response(time: Timestamp, text: Option[String])

case class Review(
  user_id: Option[String],
  name: Option[String],
  time: Timestamp,
  rating: Option[Double],
  text: Option[String],
  pics: Seq[String],
  resp: Option[Response],
  gmap_id: String
)

import org.apache.spark.sql.types._
import java.sql.Timestamp
reviewSchema: org.apache.spark.sql.types.StructType = StructType(StructField(user_id,StringType,true),StructField(name,StringType,true),StructField(time,LongType,false),StructField(rating,DoubleType,true),StructField(text,StringType,true),StructField(pics,ArrayType(StringType,true),true),StructField(resp,StructType(StructField(time,LongType,false),StructField(text,StringType,true)),true),StructField(gmap_id,StringType,false))
defined class Response
defined class Review


In [29]:
val metadataSchema = StructType(
  Seq(
    StructField("name",             StringType,                                 nullable = true),
    StructField("address",          StringType,                                 nullable = true),
    StructField("gmap_id",          StringType,                                 nullable = false),
    StructField("description",      StringType,                                 nullable = true),
    StructField("latitude",         DoubleType,                                 nullable = false),
    StructField("longitude",        DoubleType,                                 nullable = false),
    StructField("category",         ArrayType(StringType),                      nullable = true),
    StructField("avg_rating",       DoubleType,                                 nullable = false),
    StructField("num_of_reviews",   IntegerType,                                nullable = false),
    StructField("price",            StringType,                                 nullable = true),
    StructField("hours",            ArrayType(ArrayType(StringType)),           nullable = true),
    StructField("MISC",             MapType(StringType, ArrayType(StringType)), nullable = true),
    StructField("state",            StringType,                                 nullable = true),
    StructField("relative_results", ArrayType(StringType),                      nullable = true),
    StructField("url",              StringType,                                 nullable = false),
  )
)

case class Metadata(
  name: Option[String],
  address: Option[String],
  gmap_id: String,
  description: Option[String],
  latitude: Double,
  longitude: Double,
  category: Seq[String],
  avg_rating: Double,
  num_of_reviews: Int,
  price: Option[String],
  hours: Seq[Seq[String]],
  MISC: Map[String, Seq[String]],
  state: Option[String],
  relative_results: Seq[String],
  url: String
)

metadataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true),StructField(address,StringType,true),StructField(gmap_id,StringType,false),StructField(description,StringType,true),StructField(latitude,DoubleType,false),StructField(longitude,DoubleType,false),StructField(category,ArrayType(StringType,true),true),StructField(avg_rating,DoubleType,false),StructField(num_of_reviews,IntegerType,false),StructField(price,StringType,true),StructField(hours,ArrayType(ArrayType(StringType,true),true),true),StructField(MISC,MapType(StringType,ArrayType(StringType,true),true),true),StructField(state,StringType,true),StructField(relative_results,ArrayType(StringType,true),true),StructField(url,StringType,false))
defined class Metadata


### Dataset load and parse

In [30]:
import java.nio.file.Paths

val projectDir: String = Paths.get(System.getProperty("user.dir")).getParent.getParent.getParent.toString
val reviewsPath = s"$projectDir/dataset/sample-reviews.ndjson"
val metadataPath = s"$projectDir/dataset/metadata.ndjson"

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("NDJSON Reader")
  .master("local[*]") // Needed in local mode
  .getOrCreate()

val reviewsDf = spark.read
  .schema(reviewSchema)
  .json(reviewsPath)
  .withColumn("pics", when (col("pics") isNull, array()) otherwise col("pics"))
  .withColumn("time", from_unixtime(col("time") / 1000).cast("timestamp"))
  .withColumn("resp", 
    when (
      col("resp") isNotNull, 
      struct(
        from_unixtime(col("resp.time") / 1000).cast("timestamp").alias("time"),
        col("resp.text").cast(StringType).alias("text")
      )
    ) otherwise lit(null)
  )
  .as[Review]

val metadataDf = spark.read
  .schema(metadataSchema)
  .json(metadataPath)
  .withColumn("category", when (col("category") isNull, array()) otherwise col("category"))
  .withColumn("hours", when (col("hours") isNull, array()) otherwise col("hours"))
  .withColumn("relative_results", when (col("relative_results") isNull, array()) otherwise col("relative_results"))
  .withColumn("MISC",
    when (
      col("MISC") isNotNull,
      col("MISC").cast(MapType(StringType, ArrayType(StringType)))
    ) otherwise typedLit(Map.empty[String, Seq[String]])
  )
  .as[Metadata]

reviewsDf.printSchema()
metadataDf.printSchema()

// Unforturnately, it seems that Spark does not support case classes in RDDs. It throws ArrayStoreException
// when trying to collect the RDD... [see also [here](https://github.com/adtech-labs/spylon-kernel/issues/40)]
val reviewsRdd = reviewsDf.rdd
  .map(Review.unapply)
  .map(_.get)
  .map { case review @ (_, _, _, _, _, _, resp, _) => review.copy(_7 = resp.map(Response.unapply(_).get)) }
val metaRdd = metadataDf.rdd.map(Metadata.unapply).map(_.get)

root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- resp: struct (nullable = true)
 |    |-- time: timestamp (nullable = true)
 |    |-- text: string (nullable = true)
 |-- gmap_id: string (nullable = true)

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- avg_rating: double (nullable = true)
 |-- num_of_reviews: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- hours: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: 

import java.nio.file.Paths
projectDir: String = /Users/lucatassi/Projects/big-data/big-data-project
reviewsPath: String = /Users/lucatassi/Projects/big-data/big-data-project/dataset/sample-reviews.ndjson
metadataPath: String = /Users/lucatassi/Projects/big-data/big-data-project/dataset/metadata.ndjson
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7cd655ad
reviewsDf: org.apache.spark.sql.Dataset[Review] = [user_id: string, name: string ... 6 more fields]
metadataDf: org.apache.spark.sql.Dataset[Metadata] = [name: string, address: string ... 13 more fields]
reviewsRdd: org.apache.spark.rdd.RDD[(Option[String], Option[String], java.sql.Timestamp, Option[Double], Option[String], Seq[String], Option[(java.sql.Timestamp, ...


## Exploration

---

**Metadata**: (name, address, <ins>gmap_id</ins>, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url)

**Review**: (user_id, name, time, rating, text, pics, responses, <ins>gmap_id</ins>)

---

In [31]:
reviewsRdd
  .filter(_._2.contains("Hossein"))
  .collect()

res21: Array[(Option[String], Option[String], java.sql.Timestamp, Option[Double], Option[String], Seq[String], Option[(java.sql.Timestamp, Option[String])], String)] = Array()


In [32]:
metaRdd
  .filter(_._3 == "0x80dcdbd91ac0ff97:0x40cb80cf24283e4d")
  .collect()

res22: Array[(Option[String], Option[String], String, Option[String], Double, Double, Seq[String], Double, Int, Option[String], Seq[Seq[String]], Map[String,Seq[String]], Option[String], Seq[String], String)] = Array()


In [33]:
// beware `price` can be empty!!
metaRdd.filter(_._10.isEmpty).count()

res23: Long = 240140


In [34]:
// beware `rating` can be empty!!
reviewsRdd.filter(_._4.isEmpty).count()

res24: Long = 11232


In [35]:
// beware `user_id` can be empty!!
reviewsRdd.filter(_._1.isEmpty).count()

res25: Long = 11232


How many distinct businesses?

In [36]:
metaRdd.map(_._3).distinct().count()

res26: Long = 291314


How many average ratings per user? [Incomplete data since we are using a sample of the dataset]

In [37]:
val avgRatingsPerUser = reviewsRdd
  .filter(_._1.isDefined) // user_id can be not defined
  .map(_._1.get -> 1)
  .reduceByKey(_ + _) // [(user_id, #ratings written by user_id)*]
  .aggregate((0, 0))((acc, v) => (acc._1 + v._2, acc._2 + 1), (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2))
"Average ratings per user: " + avgRatingsPerUser._1.toDouble / avgRatingsPerUser._2

avgRatingsPerUser: (Int, Int) = (1809132,1151185)
res27: String = Average ratings per user: 1.5715388925324774


How many average ratings per business? [Incomplete data since we are using a sample of the dataset]

In [38]:
val avgRatingsPerBusiness = reviewsRdd
  .map(_._8 -> 1)
  .reduceByKey(_ + _) // [(gmap_id, #ratings for gmap_id)*]
  .aggregate((0, 0))((acc, v) => (acc._1 + v._2, acc._2 + 1), (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2))
"Average ratings per business: " + avgRatingsPerBusiness._1.toDouble / avgRatingsPerBusiness._2

avgRatingsPerBusiness: (Int, Int) = (1820364,193159)
res28: String = Average ratings per business: 9.424173867125011


Average response rate? [Incomplete data since we are using a sample of the dataset]

In [39]:
val avgResponseRate = reviewsRdd
  .map(r => r._8 -> (if (r._7.isDefined) 1 else 0))
  .aggregate((0, 0))((acc, v) => (acc._1 + v._2, acc._2 + 1), (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2))
"Average response rate: " + (avgResponseRate._1.toDouble / avgResponseRate._2) * 100 + " %"

avgResponseRate: (Int, Int) = (244450,1820364)
res29: String = Average response rate: 13.428632954727734 %


In [40]:
val StateRegex = """,\s*([A-Z]{2})\s+\d{5}""".r

def toState(address: Option[String]): String = address.flatMap { addr =>
    // This regex captures the state abbreviation between a comma and the ZIP code
    StateRegex.findFirstMatchIn(addr).map(_.group(1))
  }.getOrElse("Unknown")

val addr = "Redbox, 865 Thomas Rd, Warrior, AL 35180"

toState(Some(addr))

StateRegex: scala.util.matching.Regex = ,\s*([A-Z]{2})\s+\d{5}
toState: (address: Option[String])String
addr: String = Redbox, 865 Thomas Rd, Warrior, AL 35180
res30: String = AL


Spurious states are present: there exists metadata also for businesses not located in the states we are interested in.

In [48]:
val states = metaRdd.map(m => (toState(m._2), 1))
states.reduceByKey(_ + _).collect().foreach(println)

(NH,24401)
(Unknown,4728)
(WA,119096)
(AL,73872)
(MS,36658)
(NM,34133)


states: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[206] at map at <console>:40


In [42]:
val consideredStates = Map(
  "Alabama" -> "AL",
  "Mississippi" -> "MS",
  "New Hampshire" -> "NH",
  "New Mexico" -> "NM",
  "Washington" -> "WA",
)

consideredStates: scala.collection.immutable.Map[String,String] = Map(New Mexico -> NM, Washington -> WA, Alabama -> AL, Mississippi -> MS, New Hampshire -> NH)


In [43]:
println("Empty addresses: " + metaRdd.filter(_._2.isEmpty).count())
println("Spurious states: " + metaRdd.map(m => toState(m._2)).filter(s => !consideredStates.values.toSeq.contains(s)).count())

Empty addresses: 4582
Spurious states: 9143


In [47]:

// improved version
val StateNameRegex = s"""\\b(${consideredStates.keys.mkString("|")})\\b""".r
val StateAbbrevRegex = s"""\\b(${consideredStates.values.mkString("|")})\\b""".r

def toState(address: Option[String]): String = address
  .flatMap { addr =>
    StateRegex
      .findFirstMatchIn(addr)
      .map(_.group(1))
      .orElse(StateNameRegex.findFirstMatchIn(addr).map(stateName => consideredStates(stateName.group(1))))
      .orElse(StateAbbrevRegex.findFirstMatchIn(addr).map(_.group(1)))
  }
  .filter(consideredStates.values.toSeq.contains)
  .getOrElse("Unknown")

println(toState(Some("Main St, AL")))
println(toState(Some("Central Avenue, California")))
println("---")
println("Empty addresses: " + metaRdd.filter(_._2.isEmpty).count())
println("Unknown states: " + metaRdd.map(m => toState(m._2)).filter(_ == "Unknown").count())
println("Spurious states: " + metaRdd.map(m => toState(m._2)).filter(s => !consideredStates.values.toSeq.contains(s)).count())
println("---")
println("All 'unknown' have no clear state indication:")
metaRdd.map(m => m._2 -> toState(m._2)).filter(s => s._2 == "Unknown" && s._1.isDefined).collect().foreach(println)

AL
Unknown
---
Empty addresses: 4582
Unknown states: 4728
Spurious states: 4728
---
All 'unknown' have no clear state indication:
(Some(Pond Creek),Unknown)
(Some(Dry Creek),Unknown)
(Some(Lake Jackson),Unknown)
(Some(Natural Bridge Creek),Unknown)
(Some(Conecuh River),Unknown)
(Some(North America),Unknown)
(Some(Hulsey Branch),Unknown)
(Some(Limestone Creek),Unknown)
(Some(Shoe Station, 2521 Airport Thruway, Columbus, GA 31904, United States),Unknown)
(Some(Bowlero Columbus, 1636 Bradley Park Dr, Columbus, GA 31904, United States),Unknown)
(Some(Ice Queen, 6600 Whittlesey Blvd #101, Columbus, GA 31909, United States),Unknown)
(Some(Perdido Bay Seafood, 13506 Perdido Key Dr, Pensacola, FL 32507, United States),Unknown)
(Some(Walmart Supercenter, 5448 Whittlesey Blvd Ste B, Columbus, GA 31909, United States),Unknown)
(Some(FUJI JAPANESE STEAK HOUSE & SUSHI BAR, 6499 Veterans Pkwy #8, Columbus, GA 31909, United States),Unknown)
(Some(B.Merrell's, 7600 Veterans Pkwy, Columbus, GA 31909, U

StateNameRegex: scala.util.matching.Regex = \b(New Mexico|Washington|Alabama|Mississippi|New Hampshire)\b
StateAbbrevRegex: scala.util.matching.Regex = \b(NM|WA|AL|MS|NH)\b
toState: (address: Option[String])String
